# Sentence-basiertes Command-Matching testen

Dieses Notebook testet das Retrieval-basierte Command-Matching systematisch:
- Laden der CommandTemplates (502 Beispiele)
- Embedden aller Beispiel-Sätze
- Systematische Tests pro Command mit:
  - Allen Haupt-Synonymen
  - Weiteren Synonymen aus Templates
  - NEUEN Synonym-Ideen zum Testen
  - Grammatischen Formen
  - Komplexen Sätzen
- Identifikation von fehlenden Synonymen

In [1]:
import sys
sys.path.append('../src')

from utils.command_templates import COMMAND_TEMPLATES
from sentence_transformers import SentenceTransformer, util
import pandas as pd

## 1. Templates laden

In [2]:
# Übersicht
for template in COMMAND_TEMPLATES:
    print(f"{template.command:10} | {len(template.examples):2} Beispiele | Threshold: {template.threshold}")

total_examples = sum(len(t.examples) for t in COMMAND_TEMPLATES)
print(f"\nGesamt: {total_examples} Beispiele für {len(COMMAND_TEMPLATES)} Commands")

go         | 25 Beispiele | Threshold: 0.8
take       | 22 Beispiele | Threshold: 0.8
drop       | 16 Beispiele | Threshold: 0.8
use        | 26 Beispiele | Threshold: 0.8
examine    | 24 Beispiele | Threshold: 0.8
read       | 16 Beispiele | Threshold: 0.8
talk       | 21 Beispiele | Threshold: 0.8
look       | 21 Beispiele | Threshold: 0.8

Gesamt: 171 Beispiele für 8 Commands


## 2. Beispiele sammeln & embedden

In [3]:
# Model laden
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Beispiele sammeln
example_texts = []
example_commands = []
thresholds = {}

for template in COMMAND_TEMPLATES:
    for example in template.examples:
        example_texts.append(example)
        example_commands.append(template.command)
    thresholds[template.command] = template.threshold

print(f"Sammle {len(example_texts)} Beispiele...")

# Embedden
example_embeddings = model.encode(example_texts, show_progress_bar=True)

print(f"✓ {len(example_embeddings)} Embeddings erstellt")
print(f"Shape: {example_embeddings.shape}")

Sammle 171 Beispiele...


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

✓ 171 Embeddings erstellt
Shape: (171, 384)


## 3. Systematische Tests pro Command

Für jeden Command testen wir:
- Alle Haupt-Synonyme
- Verschiedene grammatische Formen
- Komplexe Sätze
- Neue Synonym-Ideen

In [ ]:
# Test-Funktion
def test_command(command_name, test_inputs):
    """Testet Command mit verschiedenen Inputs"""
    print(f"\n{'='*100}")
    print(f"TESTING: {command_name.upper()}")
    print(f"{'='*100}\n")
    
    passed = 0
    failed = 0
    
    for query in test_inputs:
        query_emb = model.encode(query)
        scores = util.cos_sim(query_emb, example_embeddings)[0]
        
        best_idx = scores.argmax().item()
        best_score = scores[best_idx].item()
        best_command = example_commands[best_idx]
        best_example = example_texts[best_idx]
        
        threshold = thresholds.get(best_command, 0.80)
        is_correct = best_command == command_name
        is_pass = best_score >= threshold
        
        if is_correct and is_pass:
            status = "✓ PASS"
            passed += 1
        elif is_correct and not is_pass:
            status = "⚠ LOW"
            failed += 1
        else:
            status = "✗ FAIL"
            failed += 1
        
        print(f"{status} | {best_score:.3f} | '{query}'")
        if not is_correct:
            print(f"         → Got: {best_command} (expected: {command_name})")
    
    print(f"\n{'='*100}")
    print(f"SUMMARY: {passed} passed, {failed} failed")
    print(f"{'='*100}\n")

## 3.1 Test GO

# Alle Synonyme testen
go_tests = [
    # Haupt-Verben
    "geh wald",
    "gehe zum wald",
    "lauf taverne",
    "laufe zur taverne",
    "renn weg",
    "renne zum ausgang",
    "komm zurück",
    "komm zum marktplatz",
    "marschiere zum schloss",
    "marschieren wald",
    
    # Weitere Synonyme (aus Templates)
    "spaziere zum see",
    "schlendere zur taverne",
    "eile zum ausgang",
    "hetze weg",
    "flüchte zum wald",
    "fliehe weg",
    "zieh zum marktplatz",
    "ziehe zur taverne",
    
    # NEUE SYNONYM-IDEEN zum Testen:
    "stürm zum wald",
    "stürme zur festung",
    "haste zum ausgang",
    "hasten weg",
    "trabe zum see",
    "traben wald",
    "schreite zur taverne",
    "schreiten vorwärts",
    "pirsch zum lager",
    "pirschen wald",
    "stapfe durch wald",
    "stapfen see",
    
    # Grammatische Formen
    "ich gehe zum wald",
    "wir gehen taverne",
    "geh du zum see",
    "zum wald gehen",
    
    # Komplexe Sätze
    "geh zur taverne und rede mit wirt",
    "lauf schnell zum ausgang",
    "kann ich zum wald gehen",
]

test_command('go', go_tests)

In [ ]:
## 3.2 Test TAKE

take_tests = [
    # Haupt-Verben
    "nimm schlüssel",
    "nimm den schlüssel",
    "hole beutel",
    "hol dir beutel",
    "pack fackel ein",
    "greif schwert",
    "greife nach schwert",
    "sammel münzen auf",
    "heb hammer auf",
    
    # Weitere Synonyme (aus Templates)
    "raff schlüssel auf",
    "schnapp schlüssel",
    "schnapp dir schlüssel",
    "klau gold",
    "steck schlüssel ein",
    "ergreif schwert",
    
    # NEUE SYNONYM-IDEEN:
    "fass schlüssel",
    "fasse schlüssel",
    "trag schlüssel",
    "trage schlüssel weg",
    "nehm schlüssel mit",
    "nehme schlüssel mit",
    "beschaff dir schlüssel",
    "beschaffen schlüssel",
    "birg schlüssel",
    "bergen schlüssel",
    "sicher schlüssel",
    "sichere dir schlüssel",
    
    # Grammatische Formen
    "ich nehme schlüssel",
    "wir nehmen den beutel",
    "nimm du schlüssel",
    "schlüssel nehmen",
    
    # Komplexe Sätze
    "nimm schlüssel und öffne truhe",
    "nimm schnell den beutel",
    "kann ich schlüssel nehmen",
]

test_command('take', take_tests)

## 3.3 Test DROP

drop_tests = [
    # Haupt-Verben
    "leg schwert ab",
    "lege schwert ab",
    "wirf fackel weg",
    "stell beutel hin",
    "lass hammer fallen",
    
    # Weitere Synonyme (aus Templates)
    "entsorge schwert",
    "platziere beutel hier",
    "deponiere hammer",
    "pack schwert aus",
    "lass schwert liegen",
    "lass schwert los",
    
    # NEUE SYNONYM-IDEEN:
    "verstaue schwert",
    "verstauen schwert",
    "setz schwert ab",
    "setze schwert ab",
    "halt schwert nicht mehr",
    "zurücklassen schwert",
    "liegenlassen schwert",
    "dalassen schwert",
    "abladen schwert",
    
    # Grammatische Formen
    "ich lege schwert ab",
    "wir werfen fackel weg",
    "leg du schwert ab",
    "schwert ablegen",
    
    # Komplexe Sätze
    "wirf schwert weg und nimm hammer",
    "leg vorsichtig hammer ab",
    "kann ich schwert ablegen",
]

test_command('drop', drop_tests)

In [ ]:
## 3.4 Test USE

use_tests = [
    # Haupt-Verben
    "benutze schlüssel",
    "verwende schlüssel",
    "nutze fackel",
    "öffne truhe",
    "bediene hebel",
    
    # Weitere Synonyme (aus Templates)
    "aktiviere hebel",
    "betätige hebel",
    "drück hebel",
    "zieh hebel",
    "schließ truhe auf",
    "mach truhe auf",
    "krieg truhe auf",
    
    # NEUE SYNONYM-IDEEN:
    "entriegle truhe",
    "entriegeln truhe",
    "entsperre truhe",
    "entsperren truhe",
    "knack truhe",
    "knacke truhe auf",
    "spreng truhe",
    "sprenge truhe auf",
    "hebel umlegen",
    "schalter betätigen",
    "mechanismus aktivieren",
    
    # Mit Werkzeug
    "öffne truhe mit schlüssel",
    "benutze schlüssel an truhe",
    
    # Grammatische Formen
    "ich benutze schlüssel",
    "wir öffnen truhe",
    "benutze du schlüssel",
    "schlüssel benutzen",
    
    # Komplexe Sätze
    "benutze schlüssel um truhe zu öffnen",
    "öffne vorsichtig truhe",
    "kann ich schlüssel benutzen",
]

test_command('use', use_tests)

## 3.5 Test EXAMINE

examine_tests = [
    # Haupt-Verben
    "untersuche truhe",
    "betrachte hammer",
    "inspiziere buch",
    "mustere inschrift",
    "prüfe mechanismus",
    "begutachte runen",
    "analysiere karte",
    "schau truhe an",
    
    # Weitere Synonyme (aus Templates)
    "check truhe",
    "checke truhe",
    "kontrolliere truhe",
    "beschau truhe",
    "beäuge truhe",
    "studiere truhe",
    "erforsche truhe",
    "durchsuche truhe",
    
    # NEUE SYNONYM-IDEEN:
    "scan truhe",
    "scanne truhe",
    "observiere truhe",
    "observieren truhe",
    "beobachte truhe",
    "beobachten truhe",
    "sichte truhe",
    "sichten truhe",
    "überprüf truhe",
    "überprüfe truhe",
    "durchleuchte truhe",
    "durchleuchten truhe",
    "beguck truhe",
    "begucke truhe",
    
    # Grammatische Formen
    "ich untersuche truhe",
    "wir betrachten truhe",
    "untersuche du truhe",
    "truhe untersuchen",
    
    # Komplexe Sätze
    "untersuche truhe genau",
    "schau truhe genau an",
    "kann ich truhe untersuchen",
]

test_command('examine', examine_tests)

In [ ]:
## 3.6 Test READ

read_tests = [
    # Haupt-Verben
    "lies buch",
    "lese buch",
    "studiere text",
    "entziffere inschrift",
    
    # Weitere Synonyme (aus Templates)
    "überfliege buch",
    "durchstöbere buch",
    "dekodiere runen",
    "entschlüssle inschrift",
    "deute runen",
    
    # NEUE SYNONYM-IDEEN:
    "verschlinge buch",
    "verschlingen buch",
    "konsultiere buch",
    "konsultieren buch",
    "nachschlagen in buch",
    "schmökere in buch",
    "schmökern buch",
    "vertiefe mich in buch",
    "vertiefen text",
    "interpretiere text",
    "interpretieren runen",
    "übersetze inschrift",
    "übersetzen runen",
    
    # Mit Partikel
    "lies buch durch",
    "lies text vor",
    
    # Grammatische Formen
    "ich lese buch",
    "wir lesen text",
    "lies du buch",
    "buch lesen",
    
    # Komplexe Sätze
    "lies buch um hinweise zu finden",
    "lies genau das buch",
    "kann ich buch lesen",
]

test_command('read', read_tests)

## 3.8 Test LOOK

In [ ]:
look_tests = [
    # Haupt-Verben
    "schau dich um",
    "schaue dich um",
    "sieh dich um",
    "guck dich um",
    "schau umher",
    "blick umher",
    "späh umher",
    
    # Weitere Synonyme (aus Templates)
    "orientiere mich",
    "kundschafte aus",
    "erkunde umgebung",
    
    # Kurze Formen
    "schau um",
    "sieh um",
    "schau",
    
    # NEUE SYNONYM-IDEEN:
    "muster umgebung",
    "mustern umgebung",
    "sondiere umgebung",
    "sondieren umgebung",
    "inspiziere umgebung",
    "inspizieren raum",
    "erfass umgebung",
    "erfassen raum",
    "wahrnehmen umgebung",
    "nimm umgebung wahr",
    "scanne raum",
    "scannen umgebung",
    "überblicke raum",
    "überblicken umgebung",
    
    # Grammatische Formen
    "ich schaue mich um",
    "wir schauen uns um",
    "schau du dich um",
    "umschauen",
    
    # Komplexe Sätze
    "schau dich genau um",
    "schau dich vorsichtig um",
    "kann ich mich umschauen",
]

test_command('look', look_tests)